## Evaluate transfer learning on Turnedtable Watertank (Dataset 2)  using an SVM Classifier: Self-supervised approach 

In [5]:
import tensorflow as tf
import h5py
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import sys
import os
sys.path.append('../../../')
sys.path.append('../../')

### Configure Tensorflow for GPU device

In [6]:
tf.config.experimental_run_functions_eagerly(True)
print("[INFO] Tensorflow Version:", tf.__version__)

if tf.config.list_physical_devices("GPU") and tf.test.is_built_with_cuda():
    print("[INFO] Tensorflow built with CUDA")
    print("[INFO] Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    print("[INFO] List of GPU devices:", tf.config.list_physical_devices("GPU"))
    physical_devices = tf.config.list_physical_devices("GPU")
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

else:
    print("[ERROR] GPU not detected, make sure tensorflow-gpu is installed and that GPU is recognized")
    exit()

[INFO] Tensorflow Version: 2.2.0
[INFO] Tensorflow built with CUDA
[INFO] Number GPUs Available:  1
[INFO] List of GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Define utilities

In [7]:
# utilities
def flatten(x):
    return x.reshape((x.shape[0], -1))

def classSampling(X, y, samplesPerClass, numberOfClasses):
    X_ret = np.zeros((samplesPerClass * numberOfClasses, X.shape[1]), dtype = np.float32)
    y_ret = np.zeros((samplesPerClass * numberOfClasses), dtype = np.uint8)
    count = 0

    for classIdx in range(numberOfClasses):
        indices = np.where(y == classIdx)[0]

        #if len(indices) < samplesPerClass:
        #    raise IndexError("Not enough samples for class {} to produce {} samples per class. Only {} class samples available".format(classIdx, samplesPerClass, len(indices)))

        doResample = len(indices) < samplesPerClass

        chosenIndices = np.random.choice(indices, samplesPerClass, replace = doResample)

        for ci in chosenIndices:
            X_ret[count] = X[ci]
            y_ret[count] = y[ci]

            count += 1

    return X_ret, y_ret

### Define dataset and dataloader

In [8]:
class SonarTurnedTableSupervised(object):
    def __init__(self, file_path):
        self.file_path = file_path

    def _normalize_images(self, images):
        """
        Normalize sonar images by 1/255.
        """
        return [element/255.0 for element in images]

    def get_sonar_data(self):
        """
        Reads from HDF5 file containing sonar data (resized to fix dims).
        Returns list of np arrays containing image data.
        """

        print("[INFO] Retrieving Sonar Turned Table Supervised Data")

        with h5py.File(self.file_path, "r") as f:
            # list all groups
            print("hdf5 dataset keys: %s" % f.keys())

            # get images and labels
            x_train = f["x_train"][...].astype(np.float32)
            y_train = f["y_train"][...]

            x_test = f["x_test"][...].astype(np.float32)
            y_test = f["y_test"][...]

            _, x_val, _, y_val = train_test_split(x_test, y_test, train_size=0.5)

            print("[INFO] Data dimensions")
            print("Train", len(x_train))
            print("Val", len(x_val))
            print("Test", len(x_test))

            # matias normalization
            # multiply by 255 because hdf5 file comes as 1/255
            x_train *= 255.0
            x_val *= 255.0
            x_test *= 255.0

            x_train -= 84.51
            x_val -= 84.51
            x_test  -= 84.51

        return (x_train, y_train), (x_val, y_val), (x_test, y_test)
    
def load_sonar_turnedtable_supervised(file_path):
    """
    Loads test data from turnedtable dataset.
    """
    print()
    print("[INFO] Loading Tenorflow dataset")

    dataset_object = SonarTurnedTableSupervised(file_path)

    # Read data
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset_object.get_sonar_data()

    # Train data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(len(x_train))
    train_dataset = train_dataset.prefetch(25)

    # Validation data
    # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, labels_val))
    # val_dataset = val_dataset.shuffle(buffer_size=len(x_val)).batch(batch_size)
    # val_dataset = val_dataset.prefetch(25)

    # Test data
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.shuffle(buffer_size=len(x_test)).batch(len(x_test)) # feed full test set
    test_dataset = test_dataset.prefetch(25)

    print()
    print("[INFO] Tensorflow data dimensions")
    # print(train_dataset)
    # print(val_dataset)
    print(test_dataset)

    # return train_dataset, val_dataset, test_dataset
    return train_dataset, test_dataset

###  Load Pretrained Model

In [9]:
from architectures.minixception import MiniXception

PRETRAINED_NUM_CLASSES = 4 # 11 supervised, 4 self-supervised
input_shape = [96, 96, 1]

model_name = "minixception"
pretraining_mode = "self_supervised_learning"
# layers = ["add_19", "add_18", "conv2d_35"]
layers = ["add_3", "add_2", "conv2d_6"]
model = MiniXception(input_shape, PRETRAINED_NUM_CLASSES)

model.summary()

Model: "MINI-XCEPTION"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 96, 8)    72          image[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 96, 96, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 96, 96, 8)    0           batch_normalization[0][0]        
______________________________________________________________________________________

In [10]:
# check weights BEFORE loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[ 0.1487208   0.04504052  0.08439493 -0.1733607  -0.05161296
    -0.12433307 -0.06567085  0.12211633]]

  [[ 0.20209298  0.14931566 -0.27014968 -0.15539922 -0.17741272
     0.20195833 -0.19076791  0.2373327 ]]

  [[-0.04493974 -0.21775824  0.04011425  0.24174488 -0.23093116
    -0.21730714  0.02564567  0.2607783 ]]]


 [[[ 0.24340802  0.12731454  0.1084058   0.01785299 -0.2543125
     0.2424931   0.05726632  0.20496836]]

  [[-0.13325982  0.23459905  0.26989496  0.07654843  0.01016757
    -0.03883523  0.01398644  0.02530792]]

  [[ 0.1232101   0.04747784 -0.04573134  0.09715047  0.13595083
     0.17142603  0.0062885  -0.08161967]]]


 [[[ 0.0943749  -0.1368835  -0.17053913  0.18167803  0.00242522
     0.06695643  0.06624961 -0.19963363]]

  [[-0.01593143  0.15150845  0.19179219 -0.08778694  0.25803113
    -0.22782576 -0.2620764   0.24952209]]

  [[ 0.10843578  0.07209289  0.06292123 -0.10812981  0.2612118
     0.11264864 -0.00313792 -0.06093465]]]]


In [11]:
# path to pretrained model checkpoint
pretrained_checkpoint_prefix = os.path.join("../../../pretraining/results/" + pretraining_mode + "/checkpoints/sonar1/" + model_name + "/batch_size_128/96x96_substract_mean_online_aug_width_16")

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.5, epsilon=1)

# define pretrained checkpoint model
checkpoint_pretrained = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model) # 4 ssl, 11 sl
manager_pretrained = tf.train.CheckpointManager(checkpoint_pretrained, pretrained_checkpoint_prefix, max_to_keep=3)

# restore model weights
checkpoint_pretrained.restore(manager_pretrained.latest_checkpoint)

if manager_pretrained.latest_checkpoint:
    print("[INFO] Pretrained checkpoint restored correctly: {}".format(manager_pretrained.latest_checkpoint))
else:
    print("[INFO] Could not restore pretrained checkpoint correctly, make sure path to pre-trained folder is correct.")

[INFO] Pretrained checkpoint restored correctly: ../../../pretraining/results/self_supervised_learning/checkpoints/sonar1/minixception/batch_size_128/96x96_substract_mean_online_aug_width_16/ckpt-21


In [12]:
# check weights AFTER loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[-0.01201695  0.10008978 -0.13219346  0.00733798 -0.07005484
    -0.04808217  0.12624699 -0.18287827]]

  [[ 0.13855451 -0.14625448 -0.21844625  0.17071846  0.09405775
    -0.24085402  0.10183581 -0.14387022]]

  [[-0.01798811 -0.02620124  0.00532504  0.24450086  0.05492317
    -0.03963403 -0.09577609 -0.17052881]]]


 [[[-0.19742756 -0.02938295  0.18144004 -0.07417174 -0.04067294
    -0.11615404  0.3142947  -0.22881457]]

  [[ 0.04655689 -0.25744978 -0.01958997 -0.2519868  -0.15674144
    -0.01226818 -0.10892429  0.05615671]]

  [[-0.1150781  -0.01644535  0.18481599  0.01680727  0.00066687
     0.02181816 -0.06617652  0.00829394]]]


 [[[-0.2902653   0.07897053 -0.22257775  0.06865413  0.2949624
     0.03776029  0.08550944  0.02152358]]

  [[ 0.04263758  0.26212874  0.15081672 -0.01832337  0.3247206
     0.24757573 -0.24490963 -0.01840872]]

  [[ 0.25585535 -0.04240311 -0.12157271 -0.16576783  0.03661726
     0.16615067 -0.18297872 -0.2156377 ]]]]


### Define models up to intermediate layers


In [13]:
print("Intermediate layers from model:", layers)

Intermediate layers from model: ['add_3', 'add_2', 'conv2d_6']


In [14]:
# https://stackoverflow.com/questions/63297838/how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction
add_3 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[0]).output)

add_2 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[1]).output)

conv2d_6 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[2]).output)


### Generate vector embeddings for train and test data (up to n-th layer)

In [15]:
# define tensorflow dataset
data_dir = "../../../../../../datasets/sonar_turntable_dataset_2/marine-debris-turntable-classification-object_classes-platform-96x96.hdf5"
train_dataset, test_dataset = load_sonar_turnedtable_supervised(data_dir)

# load tensorflow tensors individually
x_train, y_train = next(iter(train_dataset))
x_test, y_test = next(iter(test_dataset))


[INFO] Loading Tenorflow dataset
[INFO] Retrieving Sonar Turned Table Supervised Data
hdf5 dataset keys: <KeysViewHDF5 ['class_names', 'x_test', 'x_train', 'y_test', 'y_train']>
[INFO] Data dimensions
Train 1505
Val 323
Test 645

[INFO] Tensorflow data dimensions
<PrefetchDataset shapes: ((None, 96, 96, 1), (None,)), types: (tf.float32, tf.int64)>


In [16]:
# perform a forward pass to generate embeddings (both train and test data) (for each n-th layer)

# NOTE: when doing this all tensors are loaded into memory, this saturates NVIDIA memory (nvidia-smi)
x_train_add_3 = add_3([x_train], training=False)
x_train_add_2 = add_2([x_train], training=False)
x_train_conv2d_6 = conv2d_6([x_train], training=False)

x_test_add_3 = add_3([x_test], training=False)
x_test_add_2 = add_2([x_test], training=False)
x_test_conv2d_6 = conv2d_6([x_test], training=False)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Conv2D: Dst tensor is not initialized. [Op:Conv2D]

In [13]:
# x_train_add_3

<tf.Tensor: shape=(1505, 6, 6, 128), dtype=float32, numpy=
array([[[[ 2.02667928e+00,  1.25419867e+00,  3.97476506e+00, ...,
           3.90269685e+00,  1.02311695e+00,  1.33685827e-01],
         [ 7.14852035e-01,  1.28386438e+00,  1.82952583e+00, ...,
           4.01716709e+00,  3.57535839e+00, -1.15940833e+00],
         [ 4.86602724e-01,  1.33568931e+00, -4.27551389e-01, ...,
           1.64240623e+00,  5.02710342e+00, -1.94920182e+00],
         [-2.32546401e+00,  2.91767931e+00,  4.25891966e-01, ...,
           3.29664040e+00,  4.59511518e+00, -1.07874632e+00],
         [ 1.58441830e-02,  3.65500879e+00, -3.90215278e+00, ...,
           1.21211171e+00,  2.75078177e+00,  3.04705644e+00],
         [ 2.10274363e+00, -6.35860920e-01, -2.43832469e-01, ...,
           2.02246523e+00,  3.17994404e+00,  1.20501363e+00]],

        [[-5.65949535e+00,  4.10151196e+00,  4.51660252e+00, ...,
           1.78682351e+00,  4.06615782e+00,  2.90247965e+00],
         [ 2.04414740e-01,  4.59194183e+00,

### Transfer Learning setup: classification with subsamples per object class (few shot learning)

In [3]:
# transfer learning params
# SAMPLES_PER_CLASS = [1, 5, 10, 20, 30, 40, 50] # NOTE: taking more samples per class since it is 88 for 50
# SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, len(x_test)]
SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 80, 110, 140, 170, 200]
TRIALS = 10

NUM_CLASSES_WATERTANK = 11
NUM_CLASSES_TURNEDTABLE = 12

In [4]:
# Flatten train & test data for SVM
x_train_add_3 = flatten(x_train_add_3.numpy())
x_train_add_2 = flatten(x_train_add_2.numpy())
x_train_conv2d_6 = flatten(x_train_conv2d_6.numpy())

x_test_add_3 = flatten(x_test_add_3.numpy())
x_test_add_2 = flatten(x_test_add_2.numpy())
x_test_conv2d_6 = flatten(x_test_conv2d_6.numpy())

NameError: name 'flatten' is not defined

In [16]:
# these two are not modified (only x_test, x_train)
print(y_train.numpy().shape)
print(y_test.numpy().shape)

y_train = y_train.numpy() # convert from tf tensor --> numpy
y_test = y_test.numpy()

(1505,)
(645,)


### Run svm tl evaluation with spc for each n-th layer

In [17]:
def train_svm_with_spc(x_train, y_train, x_test, y_test):
    """
    Takes embeddings from pretrained model and evaluates transfer learning 
    with few samples per class.
    """
    # NOTE: svm takes original labels (not one-hot encoding)
    for spc in SAMPLES_PER_CLASS:
        accuracies = []

        for i in range(TRIALS):
            x_sample, y_sample = classSampling(x_train, y_train, spc, NUM_CLASSES_TURNEDTABLE)

            svm = SVC(C=1.0, decision_function_shape = 'ovo', kernel="linear")
            svm.fit(x_sample, y_sample)

            train_acc = svm.score(x_sample, y_sample)
            test_acc = svm.score(x_test, y_test)

            print("SPC {} Train Accuracy: {:.3f}".format(spc, train_acc))
            print("SPC {} Test Accuracy: {:.3f}".format(spc, test_acc))
            print()

            accuracies.append(test_acc)

        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)

        mean_acc = round(100 * mean_acc, 3)
        std_acc = round(100 * std_acc, 3)

        print("After {} trials - Test Accuracy is {} +- {}".format(TRIALS, mean_acc, std_acc ))
        print("------------------------------------------------------------------------------")
        print()

In [18]:
train_svm_with_spc(x_train_add_3, y_train, x_test_add_3, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.650

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.693

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.718

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.707

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.691

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.664

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.690

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.670

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.670

After 10 trials - Test Accuracy is 68.636 +- 2.135
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.783

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.792

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.771

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.764

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.769


In [19]:
train_svm_with_spc(x_train_add_2, y_train, x_test_add_2, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.668

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.656

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.673

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.674

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.668

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.657

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.616

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.662

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.687

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.710

After 10 trials - Test Accuracy is 66.713 +- 2.288
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.800

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.771

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.816

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.783

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.778


In [21]:
train_svm_with_spc(x_train_conv2d_6, y_train, x_test_conv2d_6, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.532

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.476

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.532

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.509

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.575

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.530

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.549

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.526

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.530

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.516

After 10 trials - Test Accuracy is 52.744 +- 2.434
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.598

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.639

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.623

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.558

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.606
